In [1]:
import sys
from pathlib import Path
p = Path('.').absolute().parent
if sys.path[-1] != str(p):
    sys.path.append(str(p))

from graph_irl.reward import GraphReward
from graph_irl.policy import GCN
from graph_irl.irl_trainer import IRLGraphTrainer
from graph_irl.examples import circle_graph

import numpy as np
import torch
np.random.seed(0)
torch.manual_seed(0)

from torch_geometric.data import Data, Batch

import networkx as nx

from tqdm import tqdm


# circular graph with 7 nodes;
n_nodes, node_dim = 11, 5
nodes, edge_index = circle_graph(n_nodes, node_dim)

encoder_hiddens = [7, 7, 2]
encoder = GCN(node_dim, encoder_hiddens, with_layer_norm=True, final_tanh=True)
reward_fn = GraphReward(encoder, 2, [13, 11], with_layer_norm=True)

traj_batch_size = 250



irl_trainer = IRLGraphTrainer(reward_fn, None, None, nodes, edge_index, 3, None, 4)
avg_return, rewards = irl_trainer.get_avg_expert_returns()

print(avg_return, rewards.shape)

# get single permutation of edges in expert;
perm = np.random.permutation(range(0, edge_index.shape[-1], 2))
print(len(perm), max(perm), edge_index.shape[-1], perm)
idxs = sum([(i, i + 1) for i in perm], ())
print(len(idxs), idxs, edge_index[:, idxs], sep='\n\n')

        # 
        # 
    # rewards = reward_fn()
# perm, idxs
# idxs, edge_index[:, idxs]

# G = nx.Graph()
# G.add_edges_from(list(zip(*edge_index.tolist())))
# G.add_nodes_from(range(edge_index.shape[-1] // 2))
# nx.draw_networkx(G)








tensor(-45.5960, grad_fn=<AddBackward0>) torch.Size([3, 70])
70 138 140 [ 26  24  28  36  72 100 134  12 128 116  68 110 124 118  30 120  84 104
  44  34  76  98  32  66  60 114 132   0   8  16  10   2  48  42  14 112
 130  88  62 126  52  96 106  82  90  18  78  70   6  58  40  92 102  46
  50  38  54   4  80  94  56 138  74 136 108  20  22  64  86 122]
140

(26, 27, 24, 25, 28, 29, 36, 37, 72, 73, 100, 101, 134, 135, 12, 13, 128, 129, 116, 117, 68, 69, 110, 111, 124, 125, 118, 119, 30, 31, 120, 121, 84, 85, 104, 105, 44, 45, 34, 35, 76, 77, 98, 99, 32, 33, 66, 67, 60, 61, 114, 115, 132, 133, 0, 1, 8, 9, 16, 17, 10, 11, 2, 3, 48, 49, 42, 43, 14, 15, 112, 113, 130, 131, 88, 89, 62, 63, 126, 127, 52, 53, 96, 97, 106, 107, 82, 83, 90, 91, 18, 19, 78, 79, 70, 71, 6, 7, 58, 59, 40, 41, 92, 93, 102, 103, 46, 47, 50, 51, 38, 39, 54, 55, 4, 5, 80, 81, 94, 95, 56, 57, 138, 139, 74, 75, 136, 137, 108, 109, 20, 21, 22, 23, 64, 65, 86, 87, 122, 123)

tensor([[13, 14, 12, 13, 14, 15, 18, 19, 36, 3

In [24]:
import sys
from pathlib import Path
p = Path('.').absolute().parent
if sys.path[-1] != str(p):
    sys.path.append(str(p))

from graph_irl.distributions import GaussDist, TwoStageGaussDist
from graph_irl.policy import AmortisedGaussNet, GCN, TwoStageGaussPolicy, GaussPolicy

import torch
from torch import nn
import torch.distributions as dists
from torch_geometric.data import Data, Batch


nodes = torch.randn((10, 5))
edge_index1 = torch.tensor([[], []], dtype=torch.long)
edge_index2 = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
batch = Batch.from_data_list([Data(x=nodes, edge_index=edge_index1),
                              Data(x=nodes, edge_index=edge_index2)])
encoder = GCN(5, [3, 7], with_layer_norm=True, final_tanh=False)
tsg_policy = TwoStageGaussPolicy(7, 7, [3, 4], [3, 4], encoder, with_layer_norm=True)
d, nodes = tsg_policy(batch)
actions = torch.randn((2, 14))
d.log_prob(actions[:, :7], actions[:, 7:])

tensor([[-13.2383, -11.2118,  -4.9134,  -1.9987,  -3.2452,  -2.8528,  -6.4627],
        [ -7.4922, -26.0197,  -2.4302, -21.3755,  -1.4071,  -5.9361,  -3.5627]],
       grad_fn=<AddBackward0>)

In [2]:
import numpy as np
# import torch

a = np.zeros((10, ))
max_size = len(a)
idx = 7
nums = np.ones((5, ))
idxs = (idx + np.arange(len(nums))) % max_size
idx = (idxs[-1] + 1) % max_size
print(idxs)
a[idxs] = nums
print(a, idx)

[7 8 9 0 1]
[1. 1. 0. 0. 0. 0. 0. 1. 1. 1.] 2


In [2]:
print(n_nodes, traj_batch_size)

70 250


In [4]:
# idxs = (12, 13, 4, 5, 2, 3, 6, 7, 0, 1, 10, 11, 8, 9)
pointer = 0
rewards = []
return_val = 0.
# print(n_nodes)
while pointer * traj_batch_size * 2 < edge_index.shape[-1]:
    # print(pointer * traj_batch_size)
    batch_list = []
    action_idxs = []
    for i in range(pointer * traj_batch_size * 2, 
                            min((pointer + 1) * traj_batch_size * 2, edge_index.shape[-1]), 2):
            # print(i)
            batch_list.append(
                 Data(x=nodes, edge_index=edge_index[:, idxs[pointer * traj_batch_size * 2:i]])
            )
            
            # if i == 0:
                #  print(torch.tensor([[], []], dtype=torch.long), 
                    #    edge_index[:, idxs[pointer * traj_batch_size:i]])
                 
            first, second = edge_index[0, idxs[i]], edge_index[1, idxs[i]]    
            action_idxs.append([first, second])
            # 
            #
    print(len(batch_list)) 
    # print(torch.tensor(action_idxs, dtype=torch.long))
        # ]
    # if pointer == 0:
        # temp = torch.tensor([[], []], dtype=torch.long)
        # batch_list = [Data(x=nodes, edge_index=temp)] + batch_list
    batch = Batch.from_data_list(batch_list)
    print(batch.num_graphs)
    pointer += 1
    curr_rewards = reward_fn((batch, torch.tensor(action_idxs, dtype=torch.long)), action_is_index=True)
    return_val += curr_rewards.sum()
    rewards.append(curr_rewards)
print(return_val, rewards[0].shape, torch.cat(rewards, 0))

70
70
tensor(-43.4926, grad_fn=<AddBackward0>) torch.Size([70, 1]) tensor([[-0.7020],
        [-0.3095],
        [-0.3095],
        [-0.6366],
        [-0.5339],
        [-0.9905],
        [-0.3139],
        [-0.7757],
        [-0.6957],
        [-0.5715],
        [-1.0406],
        [-0.6419],
        [-0.5616],
        [-0.3155],
        [-0.6091],
        [-0.3108],
        [-0.7229],
        [-0.6397],
        [-0.5367],
        [-0.6582],
        [-0.5253],
        [-0.6702],
        [-0.3271],
        [-0.8338],
        [-0.3276],
        [-1.0177],
        [-0.5196],
        [-0.6496],
        [-0.5722],
        [-0.6798],
        [-0.5070],
        [-0.5295],
        [-0.3305],
        [-1.1631],
        [-0.8875],
        [-0.8869],
        [-0.3288],
        [-0.8078],
        [-0.3249],
        [-0.3243],
        [-0.3239],
        [-0.8517],
        [-0.5948],
        [-0.3196],
        [-1.0673],
        [-0.9680],
        [-0.3243],
        [-0.4765],
        [-0.5214],
  

In [6]:
import torch


torch.stack([torch.arange(3), torch.arange(3, 6), torch.arange(6, 9)])

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [19]:
perm = np.random.permutation(range(0, edge_index.shape[-1], 2))
idxs = sum([(i, i + 1) for i in perm], ())# * 2 + 1
perm, idxs
idxs, edge_index[:, idxs]

((8, 9, 12, 13, 0, 1, 2, 3, 4, 5, 6, 7, 10, 11),
 tensor([[4, 5, 6, 0, 0, 1, 1, 2, 2, 3, 3, 4, 5, 6],
         [5, 4, 0, 6, 1, 0, 2, 1, 3, 2, 4, 3, 6, 5]]))

In [21]:
a = edge_index[:, :3]
edge_index, a

(tensor([[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 0],
         [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 0, 6]]),
 tensor([[0, 1, 1],
         [1, 0, 2]]))

In [23]:
edge_index[0, 0] = 0
edge_index, a

(tensor([[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 0],
         [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 0, 6]]),
 tensor([[0, 1, 1],
         [1, 0, 2]]))

In [8]:
a = [0] + torch.repeat_interleave(torch.arange(1, 7), 2).tolist() + [0]

edge_index = torch.tensor([
    a,
    ((torch.tensor([1, -1] * 7) + torch.tensor(a)) % 7).tolist()
], dtype=torch.long)

edge_index

tensor([[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 0],
        [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 0, 6]])